# Plot to verify agreement between best model and $\zeta$ Ophiuchi observable
author: [Mathieu Renzo](mailto:mrenzo@flatironinstitute.org)

In [ ]:
import sys
# the line below import stuff from git@github.com:mathren90/plotFunc.git
sys.path.append('/mnt/home/mrenzo/codes/python_stuff/plotFunc/')
from MESAreader import *
import matplotlib.pyplot as plt
%matplotlib inline
from plotDefaults import *
# plot_aux defines many auxiliary functions to make plots
from plot_aux import *

In [ ]:
set_plotDefaults()

In [ ]:
# root folder for best model
folder = "/mnt/home/mrenzo/RUNS/zeta_ophiuchi/M1_25_M2_17_P100_Z0.01/re-run_exponential/"
# folder = "/mnt/home/mrenzo/RUNS/zeta_ophiuchi/M1_30_M2_17_P100_Z0.01/"
# folder = "/mnt/home/mrenzo/RUNS/zeta_ophiuchi/M1_25_M2_17_P100_Z0.01/weak_wind/"
# folder = "/mnt/home/mrenzo/RUNS/zeta_ophiuchi/M1_25_M2_17_P100_Z0.01/do_J_accretion/"
binary = folder+"/binary_to_postRLOF//"
accretor = folder+"/accretor_to_TAMS/"
dhfile = binary+"/LOGS1/history.data"
hfile1 = binary+'/LOGS2/history.data'
hfile2 = accretor+'/LOGS/history.data'
bfile = binary+'/binary_history.data'
model_name = folder.split('/')[-2].replace('_','\_')
print(model_name)

In [ ]:
fig = plt.figure()
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:,:])

plot_HRD(ax, hfile1=hfile1, c="b", hfile2=hfile2, bfile=bfile)
zeta_oph_HRD(ax)
ax.plot(np.nan,np.nan, "ro",label=r"$\zeta \ \mathrm{Ophiuchi}$" )
ax.legend()
ax.invert_xaxis()
ax.set_xlabel(r"$\log_{10}(T_\mathrm{eff}/ [K])$")
ax.set_ylabel(r"$\log_{10}(L/L_\odot)$")
# plt.savefig("/mnt/home/mrenzo/TMP/HRD_doJaccretion.pdf")

In [ ]:
# get some numbers

# donor TAMS to RLOF delay
# get TAMS age
src, col = getSrcCol(dhfile)
h = src[:, col.index("center_h1")]
t = src[:, col.index("star_age")]*1e-3
iHdepl = h <= 1e-4
t_TAMS = min(t[iHdepl])
# get time of initial RLOF
src, col = getSrcCol(bfile)
rl_relative_gap_1 = src[:, col.index("rl_relative_overflow_1")]
t = src[:, col.index("age")]*1e-3
iRLOF = rl_relative_gap_1 > 0
t_RLOF_init = min(t[iRLOF])
print("delay donor TAMS and RLOF", f"{t_RLOF_init-t_TAMS}", "1000 yr")

# radial expansion accretor during RLOF
src, col = getSrcCol(hfile1)
radius=src[:, col.index("radius")]
print(min(radius[iRLOF]), max(radius[iRLOF]))




In [ ]:
# analize MT
fig = plt.figure(figsize=(12.5,10))
gs = gridspec.GridSpec(120, 100)
ax = fig.add_subplot(gs[:50,:94])
ax2 = fig.add_subplot(gs[60:,:94])
cax = fig.add_subplot(gs[:,95:])


# single stars in the background
root_single  = "/mnt/home/mrenzo/RUNS/zeta_ophiuchi/single_star/Z0.01"
single_star_masses = glob.glob(root_single+'/*/')
for ss in single_star_masses:
    h = ss+'/non_rot/LOGS/history.data'
    logL, logTeff = getlogLlogTeff(h)
    ax.plot(logTeff,logL, c="#808080", ls='--', lw=1, zorder=0)
    ax2.plot(logTeff,logL, c="#808080", ls='--', lw=1, zorder=0)


# for He abundances
# vmin=0.3
# vmax=1.0
# for time [in Myrs]
vmin=0
vmax=11

# accretor
logL, logTeff = getlogLlogTeff(hfile1)
src, col = getSrcCol(hfile1)
surface_he4 =  src[:, col.index('surface_he4')]
time = src[:, col.index('star_age')]*1e-6
ax2.plot(logTeff, logL, lw=1, c="#808080", zorder=0)
p = ax2.scatter(logTeff, logL, c=time, zorder=2,  vmin=vmin, vmax=vmax)

# post-binary
logL, logTeff = getlogLlogTeff(hfile2)
src, col = getSrcCol(hfile2)
surface_he4 =  src[:, col.index('surface_he4')]
time = src[:, col.index('star_age')]*1e-6
# center_he4 =  src[:, col.index('center_he4')]
ax2.plot(logTeff[0], logL[0], marker='D', c='k', ms=10)
p = ax2.scatter(logTeff, logL, c=time, zorder=2,  vmin=vmin, vmax=vmax)
fig.colorbar(p, label="$\mathrm{time\ [Myr]}$", cax=cax)
# fig.colorbar(p, label="$\mathrm{surface\ mass\ fraction}\ ^4\mathrm{He}$", cax=cax)

# donor
logL, logTeff = getlogLlogTeff(dhfile)
src, col = getSrcCol(dhfile)
surface_he4 =  src[:, col.index('surface_he4')]
time = src[:, col.index('star_age')]*1e-6
ax.plot(logTeff, logL, lw=1, zorder=0)
p = ax.scatter(logTeff, logL, c=time, zorder=2,  vmin=vmin, vmax=vmax)

# ax.set_title(model_name, size=30)
zeta_oph_HRD(ax2)

ax.set_xlim(xmax=4.625,xmin=3.85)
ax.set_ylim(4.8,5.6)
ax2.set_xlim(xmax=4.55,xmin=4.33)
ax.invert_xaxis()
ax2.invert_xaxis()
ax2.set_xlabel(r"$\log_{10}(T_\mathrm{eff}/ [K])$")
ax.set_ylabel(r"$\log_{10}(L/L_\odot)$")
ax2.set_ylabel(r"$\log_{10}(L/L_\odot)$")



fig.align_labels()
fig.savefig("/mnt/home/mrenzo/TMP/HRD_both_age.pdf")

In [ ]:
# mass transfer rate
fig = plt.figure(figsize=(15,9))
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[50:,:])
bx = fig.add_subplot(gs[:50,:])

src, col = getSrcCol(bfile)
lg_mtransfer_rate = src[:, col.index('lg_mtransfer_rate')]
lg_mstar_dot_1 = src[:, col.index('lg_mstar_dot_1')]
lg_mstar_dot_2 = src[:, col.index('lg_mstar_dot_2')]
lg_system_mdot_1 = src[:, col.index('lg_system_mdot_1')]
lg_system_mdot_2 = src[:, col.index('lg_system_mdot_2')]
lg_wind_mdot_1 = src[:, col.index('lg_wind_mdot_1')]
lg_wind_mdot_2 = src[:, col.index('lg_wind_mdot_2')]
t = src[:, col.index("age")]*1e-4
# t = np.log10(t[-1]-t)

ax.plot(t, lg_mtransfer_rate, lw=5, c='c')
bx.plot(t, lg_mtransfer_rate, lw=5, c='c', label="$\dot M_\mathrm{Kolb}$")

ax.plot(t, lg_mstar_dot_2, lw=5, ls='--', c='b')
bx.plot(t, lg_mstar_dot_1, lw=5, ls='--', c='b', label="$\dot M_\star$")

ax.plot(t, lg_wind_mdot_2, c='r')
bx.plot(t, lg_wind_mdot_1, c='r', label=r"$\dot M_\mathrm{wind}$")

bx.legend(ncol=3)

ax.text(0.05,0.8, "$\mathrm{Accretor}$", fontsize=30, transform=ax.transAxes)
bx.text(0.05,0.8, "$\mathrm{Donor}$", fontsize=30, transform=bx.transAxes)



ax.set_ylabel(r"$\log_{10}(|\dot{M}|/\mathrm{[M_\odot \ yr^{-1}]})$",  horizontalalignment='center', y=1.)
#ax.set_xlim(xmin=5.72, xmax=5.74)
# ax.set_xlabel(r"$\log_{10}(t_\mathrm{end}-t \ \mathrm{[yr]})$")
ax.set_xlabel(r"$\mathrm{time} \ \mathrm{[10^4\,yr]}$")
bx.set_xlabel("")
bx.set_xticklabels([])
# ax.set_xlim(xmin=7.25e2, xmax=7.27e2)
ax.set_ylim(-8.5,0.3)
bx.set_xlim(ax.get_xlim())
bx.set_ylim(ax.get_ylim())
# plt.savefig("/mnt/home/mrenzo/TMP/MT.pdf")



In [ ]:
# chemical composition
fig = plt.figure()
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:,:])

final_h1, final_he4, final_n14, final_c12, final_o16 = plot_surface_abundances(hfile1, hfile2, ax=ax)
ax.legend(handlelength=0.5, columnspacing=0.75, handletextpad=0.5, ncol=3, loc="best", fontsize=20)
ax.set_yscale('log')
ax.set_xlim(0,11.7)
print(f"{final_h1:0.5f}", f"{final_he4:0.5f}", f"{final_n14:0.5f}", f"{final_c12:0.5f}", f"{final_o16:0.5f}")
# plt.savefig("/mnt/home/mrenzo/TMP/composition_zeta.pdf")

In [ ]:
# surface rotation rate
fig = plt.figure(figsize=(10,3))
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:,:])

plot_surface_rotation_time(ax, hfile1, c='k', hfile2=hfile2) #, label="Z=0.01")
# t, v = get_surface_rotation_time(hfile1)
# src, col = getSrcCol(hfile1)
# modnum = src[:, col.index("model_number")]
# ax.scatter(t, v, c=modnum, zorder=2, vmin=vmin, vmax=vmax)
# t, v = get_surface_rotation_time(hfile2)
# src, col = getSrcCol(hfile2)
# modnum = src[:, col.index("model_number")]

# ax.scatter(t, v, c=modnum, zorder=2, vmin=vmin, vmax=vmax)
ax.plot(np.nan, np.nan,c='k', ls="-.", label="$\mathrm{as\ single}$")
# overplot observational range
vsini, err_vsini = get_zeta_oph_vsini()
ax.set_xlim(0, 15.3)
xmin, xmax = ax.get_xlim()
ax.fill_between(np.linspace(xmin, xmax,2), [vsini-err_vsini], [vsini+err_vsini], facecolor='r', alpha=0.5, zorder=1)
ax.fill_between(np.linspace(xmin, xmax,2), [vsini-5*err_vsini], [vsini+3*err_vsini], facecolor='r', alpha=0.2, zorder=1)
ax.legend(loc="center left", fontsize=20)
ax.set_xlabel(r"$\mathrm{time \ [Myr]}$")
# ax.set_xlim(xmin=8)
# ax.set_ylim(ymin=200)
ax.set_ylabel(r"$v_\mathrm{rot} \ \mathrm{[km\ s^{-1}]}$")
# plt.savefig("/mnt/home/mrenzo/TMP/zeta_rot.pdf")

In [ ]:
# orbital evolution
MassVelocityEvolution(binary, figName="")

N.B: The kippenhahn diagrams do not show rotational mixing!!

In [ ]:
# Kippenhahn diagram
fig = plt.figure(figsize=(15,9))
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:,:])

x = 'model_number' # 'star_age' #
y =  'radius' # 'mass' #

mmm.loadHistory(f=binary+'/LOGS1')
ppp.plotKip3(mmm, fig=fig, ax=ax,
             xaxis=x,
             yaxis=y,
             reloadHistory=False,
             show_mass_loc=False,
             show_mix_labels=True,
             show=False,
             mix_alpha=0.5,# mix_hatch='//', hatch_color='#98ef56',
             # age_lookback=True,
             age_log=False,
             zone_frac = 10, # 0.2,
             colorbar=False,
             cmin=-12.45, cmax=12.5)

if y=='radius':
    ax.set_ylim(ymax=200)

# plt.savefig("/mnt/home/mrenzo/TMP/kipp_mass.pdf")

In [ ]:
fig = plt.figure(figsize=(15,9))
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:,:])

x = 'model_number' # 'star_age' #
y = 'mass' # 'radius' #

mmm.loadHistory(f=binary+'/LOGS1/')
ppp.plotKip3(mmm, fig=fig, ax=ax,
             xaxis=x,
             yaxis=y,
             reloadHistory=False,
             show_mass_loc=False,
             show_mix_labels=True,
             show=False,
             mix_alpha=0.5,# mix_hatch='//', hatch_color='#98ef56',
             # age_lookback=True,
             age_log=False,
             zone_frac = 10, # 0.2,
             colorbar=False,
             cmin=-12.45, cmax=12.5)

if y=='radius':
    ax.set_ylim(ymax=40)
ax.set_ylim(ymin=12, ymax=13)

In [ ]:
src, col = getSrcCol(hfile2)
mix_type1 = src[:, col.index("mix_type_1")]
mix_type2 = src[:, col.index("mix_type_2")]
mix_type3 = src[:, col.index("mix_type_3")]
mix_type4 = src[:, col.index("mix_type_4")]
mix_type5 = src[:, col.index("mix_type_5")]
mix_type6 = src[:, col.index("mix_type_6")]
mix_type7 = src[:, col.index("mix_type_7")]
mix_type8 = src[:, col.index("mix_type_8")]
mix_type9 = src[:, col.index("mix_type_9")]
mix_type10 = src[:, col.index("mix_type_10")]
mix_type11 = src[:, col.index("mix_type_11")]
mix_type12 = src[:, col.index("mix_type_12")]
mix_type13 = src[:, col.index("mix_type_13")]
mix_type14 = src[:, col.index("mix_type_14")]
mix_type15 = src[:, col.index("mix_type_15")]
mix_type16 = src[:, col.index("mix_type_16")]
mix_type17 = src[:, col.index("mix_type_17")]
mix_type18 = src[:, col.index("mix_type_18")]
mix_type19 = src[:, col.index("mix_type_19")]
mix_type20 = src[:, col.index("mix_type_20")]
t = src[:, col.index("model_number")]

fig = plt.figure(figsize=(15,9))
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:,:])


ax.scatter(t,mix_type1)
ax.scatter(t,mix_type2)
ax.scatter(t,mix_type3)
ax.scatter(t,mix_type4)
ax.scatter(t,mix_type5)
ax.scatter(t,mix_type6)
ax.scatter(t,mix_type7)
ax.scatter(t,mix_type8)
ax.scatter(t,mix_type9)
ax.scatter(t,mix_type10)
ax.scatter(t,mix_type11)
ax.scatter(t,mix_type12)
ax.scatter(t,mix_type13)
ax.scatter(t,mix_type14)
ax.scatter(t,mix_type15)
ax.scatter(t,mix_type16)
ax.scatter(t,mix_type17)
ax.scatter(t,mix_type18)
ax.scatter(t,mix_type19)
ax.scatter(t,mix_type20)

ax.set_xlabel(r"model number")
ax.set_ylabel(r"mix type")
ax.axhline(5)



In [ ]:
# spectroscopic HRD
fig = plt.figure()
gs = gridspec.GridSpec(100, 100)
ax = fig.add_subplot(gs[:,:])
plot_spectroscopic_HRD(ax, hfile1, c='k', hfile2=hfile2, label=model_name)
zeta_oph_spectroscopicHRD(ax)
ax.legend(fontsize=20)
ax.invert_xaxis()
ax.invert_yaxis()
ax.set_xlabel(r"$\log_{10}(T_\mathrm{eff}/ [K])$")
ax.set_ylabel(r"$\log_{10}(g/\mathrm{cm\ s^{-2}})$")